In [19]:
import tkinter as tk
import datetime
from dateutil import parser
from pymongo import MongoClient
from passlib.context import CryptContext
from pprint import pprint
from bson import ObjectId
import queries as qr

In [2]:
client = MongoClient(port=27017)
db = client.TransportDept
pwd_context = CryptContext(schemes=["sha512_crypt"], 
                           default="sha512_crypt", 
                           sha512_crypt__default_rounds=45000)

In [29]:
eid=0

class SampleApp(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self._frame = None
        self.switch_frame(HomePage)

    def switch_frame(self, frame_class):
        """Destroys current frame and replaces it with a new one."""
        new_frame = frame_class(self)
        if self._frame is not None:
            self._frame.destroy()
        self._frame = new_frame
        self._frame.pack()


class HomePage(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)

        start_label = tk.Label(self, text="Transport Department\n")
        signin_button = tk.Button(self, text="Sign In",
                                  command=lambda: master.switch_frame(SignIn))
        start_label.pack(side="top", fill="x", pady=10)
        signin_button.pack()


class SignIn(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)

        start_label = tk.Label(self, text="Enter the following credentials")
        
        eid_label = tk.Label(self, text="Employee Id")
        self.eid_entry = tk.Entry(self)
        password_label = tk.Label(self, text="Password")
        self.password_entry = tk.Entry(self,show="*")
        start_button = tk.Button(self, text="Login",
                                 command=lambda: self.callback(master))
        start_label.pack(side="top", fill="x", pady=10)
        eid_label.pack()
        self.eid_entry.pack()
        password_label.pack()
        self.password_entry.pack()
        start_button.pack()
    def callback(self,master):
        global eid
        eid=self.eid_entry.get()
        eid = int(eid)
        pwd=self.password_entry.get()
        val,typeemp=qr.auth(eid,pwd,db)
        if val:
            master.switch_frame(UserMenu)
        else:
            end_label = tk.Label(self,text=typeemp)
            end_label.pack()
class UserMenu(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)

        emp = db.employee.find_one({'eid':eid})
        name_of_emp = "Welcome "+ emp['FirstName']
        start_label = tk.Label(self, text=name_of_emp)
        Book_button = tk.Button(self, text="Book a car",
                                  command=lambda: master.switch_frame(Booking))
        VC_button = tk.Button(self, text="View/Cancel",
                                  command=lambda: master.switch_frame(VCPage))
        IncreaseBookingTime_button = tk.Button(self, text="Increase Booking Duration",
                                  command=lambda: master.switch_frame(IncreaseBookingTime))
        view_cur_trip_button = tk.Button(self, text="View current trip details",
                                  command=lambda: master.switch_frame(CurTrip))
        Logout_button = tk.Button(self, text="Logout",
                                  command=lambda: master.switch_frame(HomePage))
        alert_label = tk.Label(self, text="Alerts\n")
        start_label.pack(side="top", fill="x", pady=10)
        Book_button.pack()
        VC_button.pack()
        IncreaseBookingTime_button.pack()
        view_cur_trip_button.pack()
        Logout_button.pack()
        alert_label.pack()


class Booking(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)

        start_label = tk.Label(self, text="Enter the following -")
        
        date_label = tk.Label(self, text="Date(DD-MM-YYY)")
        self.date_entry = tk.Entry(self)
        time_label = tk.Label(self, text="Time(in hours from 0-23)")
        self.time_entry = tk.Entry(self)
        noofpeople_label = tk.Label(self, text="No. of people")
        self.noofpeople_entry = tk.Entry(self)
        hiretime_label = tk.Label(self, text="Hire Time")
        self.hiretime_entry = tk.Entry(self)
        destination_label = tk.Label(self, text="Destination")
        self.destination_entry = tk.Entry(self)
        distance_label = tk.Label(self, text="Distance")
        self.distance_entry = tk.Entry(self)
        waitingtime_label = tk.Label(self, text="Waiting time")
        self.waitingtime_entry = tk.Entry(self)
        reason_label = tk.Label(self, text="Reason for hire")
        self.reason_entry = tk.Entry(self)
        ctype_label = tk.Label(self, text="Car type")
        self.ctype_entry = tk.Entry(self)
        
        start_button = tk.Button(self, text="Book",
                                 command=lambda: self.callback(master))
        start_label.pack(side="top", fill="x", pady=10)
        
        date_label.pack()
        self.date_entry.pack()
        time_label.pack()
        self.time_entry.pack()
        noofpeople_label.pack()
        self.noofpeople_entry.pack()
        hiretime_label.pack()
        self.hiretime_entry.pack()
        destination_label.pack()
        self.destination_entry.pack()
        distance_label.pack()
        self.distance_entry.pack()
        waitingtime_label.pack()
        self.waitingtime_entry.pack()
        reason_label.pack()
        self.reason_entry.pack()
        ctype_label.pack()
        self.ctype_entry.pack()
        start_button.pack()
        
    def callback(self,master):
        date=self.date_entry.get()
        #print(eid)
        
        
        time=self.time_entry.get()
        dt=parser.parse(date,dayfirst=True) + datetime.timedelta(hours=int(time))
        print(dt)
        
        noofpeople=self.noofpeople_entry.get()
        
        hiretime=self.hiretime_entry.get()
        
        destination=self.destination_entry.get()
        
        distance=self.distance_entry.get()
        
        waitingtime=self.waitingtime_entry.get()
        
        reason=self.reason_entry.get()
        
        ctype=self.ctype_entry.get()
        msg=qr.create_new_booking(dt,int(noofpeople),int(eid),int(hiretime),destination,int(distance),int(waitingtime),reason,int(ctype),db)
        print(msg)
        master.switch_frame(UserMenu)
class VCPage(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)
        
        
        start_label = tk.Label(self, text="Pending Bookings")
        #vals = #function in backend to retrieve all pending bookings for emp
        
        vals = qr.retrive_next_week_booking(eid,40,db)
       
        result = {} 
        for v in vals:
            result[v['_id']]=tk.IntVar()
            #label = tk.Label(self, text= v['DateTime'])
            dt = v['Date Time'].strftime('%d-%m-%y %H:00:00')
            cb = tk.Checkbutton(self,text="Date and Time :" + dt +" Duration:"+str(v['Hire Time'])+" hrs",variable=result[v['_id']])
            cb.pack()  
            
        
        start_label.pack()
        
        submit_button = tk.Button(self, text="Submit",command=lambda: self.cancel(master,result))
        #back_button = tk.Button(self, text="Back",command=lambda: master.switch_frame(UserMenu))
        #self.booking1_checkbutton =tk.Checkbutton(self,text="Booking 1")
        #self.booking2_checkbutton =tk.Checkbutton(self,text="Booking 2")
        submit_button.pack()
        
        
        #start_label.pack(side="top", fill="x", pady=10)
        #back_button.pack(side="bott
        #om", fill="x", pady=10)
        #cancel_button.pack(side="bottom", fill="x", pady=10)
        #if(var.get()==0):
        #self.booking1_checkbutton.pack()
        #self.booking2_checkbutton.pack() 
        
    def cancel(self,master,result):
        #function in backend
        #return result to function in backend. delete all bookings with bid=key and value =1
        for r in result:
            if int(result[r].get())==1:
                qr.cancel_one_booking(r,db)
        master.switch_frame(BookingCancelled)
        
class BookingCancelled(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)

        start_label = tk.Label(self, text="Booking Cancelled Successfully")
        done_button = tk.Button(self, text="Back to User Menu",
                                  command=lambda: master.switch_frame(UserMenu))
        start_label.pack()
        done_button.pack()

class CurTrip(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)

        start_label = tk.Label(self, text="Current Trip Details")
        done_button = tk.Button(self, text="Back to User Menu",
                                  command=lambda: master.switch_frame(UserMenu))
        details = qr.retrive_cur_trip_details(eid,db)
        
        if details == {}:
            no_trip_label = tk.Label(self, text="Currently not on any trip")
            start_label.pack()
            no_trip_label.pack()
            done_button.pack()
            return
            
        car_label = tk.Label(self, text="Car Registration Number:"+str(details['Car Reg No']))
        driver_label = tk.Label(self, text="Driver's Name:"+details['Driver Name'])
        driver_mob_label = tk.Label(self, text="Driver's Mobile Number:"+str(details['Driver Mobile']))

        start_label.pack()
        car_label.pack()
        driver_label.pack()
        driver_mob_label.pack()
        done_button.pack()
        
class IncreaseBookingTime(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)
        
        
        start_label = tk.Label(self, text="Pending Bookings")
        #vals = #function in backend to retrieve all pending bookings for emp
        
        vals = qr.retrive_next_week_booking(eid,40,db)
       
        result = tk.StringVar()
        for v in vals:
            dt = v['Date Time'].strftime('%d-%m-%y %H:00:00')
            rb = tk.Radiobutton(self,text="Date and Time :" + dt+" Duration"+str(v['Hire Time'])+" hrs",variable=result,value=v['_id'])
            rb.pack()  
            
        timedur_label = tk.Label(self, text="Time(in hours from 0-23)")
        self.timedur_entry = tk.Entry(self)
        timedur_label.pack()
        self.timedur_entry.pack()
        start_label.pack()
        submit_button = tk.Button(self, text="Submit",command=lambda: self.inrdur(master,result))
        submit_button.pack()

    def inrdur(self,master,result):
        if qr.increase_booking_time(ObjectId(result.get()),int(self.timedur_entry.get()),db):
            master.switch_frame(DurationIncreased)
        else:
            master.switch_frame(DurationNotIncreased) 

class DurationIncreased(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)

        start_label = tk.Label(self, text="Booking Time increased successfully")
        done_button = tk.Button(self, text="Back to User Menu",
                                  command=lambda: master.switch_frame(UserMenu))
        start_label.pack()
        done_button.pack()
class DurationNotIncreased(tk.Frame):
    def __init__(self, master):
        tk.Frame.__init__(self, master)

        start_label = tk.Label(self, text="Sorry! We could not extend your booking time as Driver or Car not free")
        done_button = tk.Button(self, text="Back to User Menu",
                                  command=lambda: master.switch_frame(UserMenu))
        start_label.pack()
        done_button.pack()
        
        
if __name__ == "__main__":
    app = SampleApp()
    app.mainloop()